<a href="https://colab.research.google.com/github/MorielDev/TensorFlow/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT ALL THE NECESSARY LIBRARIES**

In [2]:
%tensorflow_version 2.x
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
!pip install --upgrade keras


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# **LOAD THE DATASET FOR BOTH TRAIN AND TEST**

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Load the dataset
train_dataset = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_dataset = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Display the first few rows of the train dataset
print(train_dataset.head())

--2024-06-11 12:27:08--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-06-11 12:27:08 (14.7 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-06-11 12:27:08--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

# **PREPROCESS THE DATA**

In [4]:
# Convert labels to binary values
train_dataset['label'] = train_dataset['label'].map({'ham': 0, 'spam': 1})
test_dataset['label'] = test_dataset['label'].map({'ham': 0, 'spam': 1})

# Prepare the tokenizer on training data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_dataset['message'])
train_sequences = tokenizer.texts_to_sequences(train_dataset['message'])
test_sequences = tokenizer.texts_to_sequences(test_dataset['message'])

# Pad the sequences
max_length = 100
X_train = pad_sequences(train_sequences, maxlen=max_length)
X_test = pad_sequences(test_sequences, maxlen=max_length)

# Labels
y_train = train_dataset['label'].values
y_test = test_dataset['label'].values


In [5]:
# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    LSTM(units=64, return_sequences=False),
    Dropout(rate=0.5),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.8817 - loss: 0.3446 - val_accuracy: 0.9821 - val_loss: 0.0709
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9882 - loss: 0.0516 - val_accuracy: 0.9868 - val_loss: 0.0518
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9975 - loss: 0.0149 - val_accuracy: 0.9821 - val_loss: 0.0619
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9990 - loss: 0.0059 - val_accuracy: 0.9916 - val_loss: 0.0358
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9996 - loss: 0.0038 - val_accuracy: 0.9940 - val_loss: 0.0362
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9987 - loss: 0.0048 - val_accuracy: 0.9940 - val_loss: 0.0377
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9999 - loss: 0.0010 - val_accuracy: 0.9892 - val_loss: 0.0527
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9992 - loss: 0.0021 - val_accuracy: 0

In [7]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy}")


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9872 - loss: 0.0721
Test Accuracy: 0.9885057210922241


In [8]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Preprocess the input message
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)

    # Predict the probability of the message being spam
    prediction = model.predict(padded_sequence)[0][0]

    # Determine the label based on the prediction
    label = 'spam' if prediction > 0.5 else 'ham'

    return (prediction, label)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
(5.6319772e-05, 'ham')


In [9]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
You passed the challenge. Great job!
